<a href="https://colab.research.google.com/github/peremartra/Tailoring-LLM-Architectures/blob/CH06/CH06/CH06_NB01_Data_Cosmopedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tailoring LLM Architectures**
## **Chapter 6: The Fuel — Data Strategy for Knowledge Recovery**

### **Notebook 1: Quality vs Quantity (Cosmopedia)**
by [Pere Martra](https://github.com/peremartra)

[![LinkedIn](https://img.shields.io/badge/LinkedIn-0077B5?style=flat&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/pere-martra/) [![GitHub](https://img.shields.io/badge/GitHub-100000?style=flat&logo=github&logoColor=white)](https://github.com/peremartra) [![Hugging Face](https://img.shields.io/badge/🤗%20Hugging%20Face-blue)](https://huggingface.co/oopere)

___

**Colab Environment:** GPU T4  
- **Models recommended:** google/gemma-3-270m  
- **Tested with:** meta-llama/Llama-3.2-1B using a **GPU A100**

---

In this notebook, we challenge the standard. We will use only 15,000 high-quality data samples (Cosmopedia) to attempt to beat the results we obtained in Chapter 2 with 30,000 web crawl samples (SlimPajama).

**What we'll accomplish:**
- **Load our models**: Original (teacher) and pruned (student) (Reusing logic from Ch2)
- **Prepare High-Quality Data**: Load synthetic textbooks from Cosmopedia
- **Apply Knowledge Distillation**: Train the pruned model with high-quality data
- **Measure recovery**: Compare against the baseline set in Chapter 2


In [1]:
# Libraries
# Install required packages
!pip install -q transformers torch optipfair datasets accelerate sentencepiece lm-eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 8.0 MB/s eta 0:00:00


In [2]:
# --- BASELINE RESULTS (FROM CHAPTER 2) ---
# We use the results obtained in CH02_NB02 as our baseline to beat.
# Dataset: SlimPajama (Web Crawl)
# Size: 30,000 samples
# Training: 3 Epochs
baseline_metrics = {
    "name": "SlimPajama (15k samples)",
    "arc_easy": 0.5200,      # Value from Ch2
    "hellaswang": 0.4100,      # Value from Ch2
    "lambada_openai": 0.3900,    # Value from Ch2
    "winogrande": 0.5800          # Value from Ch2
}

print(f"🎯 Target to beat: ARC-Easy {baseline_metrics['arc_easy']:.2%}")

🎯 Target to beat: ARC-Easy 52.00%


In [3]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.optim import AdamW
from optipfair import prune_model
from datasets import load_dataset
from torch.nn import functional as F
from torch.utils.data import DataLoader
from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM
import time
import json
from typing import Dict, List, Any
import copy

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU: NVIDIA L4


In [4]:
import random
import numpy as np
import torch
import os
from transformers import set_seed

def set_reproducibility(seed=42):
    # 1. Semilla para Python y librerías básicas
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    # 2. Semilla para PyTorch (CPU y GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # Para multi-GPU

    # 3. Configurar cuDNN para que sea determinista
    # Nota: Esto puede ralentizar un poco el entrenamiento
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # 4. Semilla para Transformers (Hugging Face)
    set_seed(seed)

    print(f"✅ Seed {seed} established.")

set_reproducibility(42)

✅ Seed 42 established.
